# Get fluency scores from WhisperT output

This is a script to explore to what extent we can get fluency scores from WhisperT output.

In [4]:
import pandas as pd
import json
import glob
import os

In [8]:
### Data

# WhisperT_dis json results
json_whispert_round1_dir = '/vol/tensusers2/wharmsen/SERDA-experiment-data/round1/stories/whispert_dis/json-asr-results'
json_whispert_round2_dir = '/vol/tensusers2/wharmsen/SERDA-experiment-data/round2/stories/whispert_dis/json-asr-results'

json_whispert_round1_files = glob.glob(os.path.join(json_whispert_round1_dir, '*.json'))

In [29]:
json_whispert_round1_files[0]

'/vol/tensusers2/wharmsen/SERDA-experiment-data/round1/stories/whispert_dis/json-asr-results/5NMJH-story_3-20221107131303195.json'

In [25]:
# Read WhisperT json file

def whisperTOutputJsonToDict(jsonFile):
    with open(jsonFile, 'r') as f:
        data = json.load(f)
    return data

file_object = whisperTOutputJsonToDict(json_whispert_round1_files[0])
segments = file_object['segments']
nrSegments = len(segments)

wordsFromSegments = [segment['words'] for segment in segments]
words = [item for sublist in wordsFromSegments for item in sublist]
wordsDF = pd.DataFrame(words)
wordsDF.head(20)

,text,start,end,confidence
0,[*],0.00,1.44,0.000
1,Verslaafd.,1.44,2.34,0.729
2,[*],2.76,3.16,0.000
3,Als,3.16,3.46,0.528
4,[*],3.46,4.02,0.000
5,ouders,4.02,4.36,0.831
6,zien,4.36,4.56,0.857
7,dat,4.56,4.84,0.899
8,je,4.84,5.16,0.863
9,[*],5.16,5.40,0.000


In [28]:
wordsDf_without_dis = wordsDF[wordsDF['text'] != '[*]']
wordsDf_without_dis

,text,start,end,confidence
1,Verslaafd.,1.44,2.34,0.729
3,Als,3.16,3.46,0.528
5,ouders,4.02,4.36,0.831
6,zien,4.36,4.56,0.857
7,dat,4.56,4.84,0.899
...,...,...,...,...
186,je,100.58,100.74,0.852
187,aan,100.74,100.86,0.816
188,het,100.86,101.02,0.883
189,spelen,101.02,101.38,0.847


In [52]:
aligned_csv = '/vol/tensusers2/wharmsen/SERDA-experiment-data/round1/stories/whispert_dis/csv-alignments/5NMJH-story_3-20221107131303195.csv'
aligned_df = pd.read_csv(aligned_csv)
aligned_df['sentenceID'] = [int(x.split('-')[0]) for x in aligned_df['promptID']]
aligned_df['wordID'] = [int(x.split('-')[1]) for x in aligned_df['promptID']]
aligned_df

,promptID,aligned_asrTrans,reversed_aligned_asrTrans,correct,confidence,startTime,endTimes,sentenceID,wordID
0,0-0-Verslaafd,verslaafd,verslaafd,True,0.729,1.44,2.34,0,0
1,1-0-Als,als,als,True,0.528,3.16,3.46,1,0
2,1-1-ouders,ouders,ouders,True,0.831,4.02,4.36,1,1
3,1-2-zien,zien,zien,True,0.857,4.36,4.56,1,2
4,1-3-dat,dat,dat,True,0.899,4.56,4.84,1,3
...,...,...,...,...,...,...,...,...,...
167,11-6-je,***,**e,False,0.000,0.00,0.00,11,6
168,11-7-aan,****,***n,False,0.000,0.00,0.00,11,7
169,11-8-het,****,****,False,0.000,0.00,0.00,11,8
170,11-9-spelen,spel**,*******,False,0.000,0.00,0.00,11,9


In [54]:
aligned_df[aligned_df['wordID'] == 0]

,promptID,aligned_asrTrans,reversed_aligned_asrTrans,correct,confidence,startTime,endTimes,sentenceID,wordID
0,0-0-Verslaafd,verslaafd,verslaafd,True,0.729,1.44,2.34,0,0
1,1-0-Als,als,als,True,0.528,3.16,3.46,1,0
21,2-0-Jij,jij,*jij,True,0.885,14.86,15.40,2,0
39,3-0-Hier,hier,*hier,True,0.513,24.76,24.94,3,0
50,4-0-Game,g***,ga**,False,0.000,0.00,0.00,4,0
56,5-0-Game,game,game,True,0.360,37.22,37.58,5,0
63,6-0-Leg,leg,leg,True,0.485,40.34,40.54,6,0
98,7-0-Vertel,vert**,*vertel,True,0.830,59.16,59.62,7,0
115,8-0-Vertel,*******,*******,False,0.000,0.00,0.00,8,0
141,9-0-Natuurlijk,natuurlijk,**atuurlijk,True,0.626,91.30,91.94,9,0


In [56]:
maxSentenceID = max(set(aligned_df['sentenceID']))
maxSentenceID
#aligned_df[aligned_df['wordID'] == '0']

11

In [59]:
for sentenceID in range(12):
    setje = aligned_df[aligned_df['sentenceID'] == sentenceID]
    print(setje)

        promptID aligned_asrTrans reversed_aligned_asrTrans  correct  \
0  0-0-Verslaafd        verslaafd                 verslaafd     True   

   confidence  startTime  endTimes  sentenceID  wordID  
0       0.729       1.44      2.34           0       0  
          promptID aligned_asrTrans reversed_aligned_asrTrans  correct  \
1          1-0-Als              als                       als     True   
2       1-1-ouders           ouders                    ouders     True   
3         1-2-zien             zien                      zien     True   
4          1-3-dat              dat                       dat     True   
5           1-4-je               j*                        je     True   
6         1-5-veel            ***e*                      ****    False   
7          1-6-aan              aan                      *aan     True   
8          1-7-het              het                       het     True   
9        1-8-gamen            gamen                     gamen     True   
1

In [60]:
setje

,promptID,aligned_asrTrans,reversed_aligned_asrTrans,correct,confidence,startTime,endTimes,sentenceID,wordID
161,11-0-Misschien,********en,*of******,False,0.000,0.00,0.00,11,0
162,11-1-wel,*of,***,False,0.000,0.00,0.00,11,1
163,11-2-over,over,*over,True,0.804,98.98,99.40,11,2
164,11-3-het,****,***,False,0.000,0.00,0.00,11,3
165,11-4-spel,*****,*spel,True,0.684,99.58,99.92,11,4
166,11-5-dat,****,****,False,0.000,0.00,0.00,11,5
167,11-6-je,***,**e,False,0.000,0.00,0.00,11,6
168,11-7-aan,****,***n,False,0.000,0.00,0.00,11,7
169,11-8-het,****,****,False,0.000,0.00,0.00,11,8
170,11-9-spelen,spel**,*******,False,0.000,0.00,0.00,11,9
